### Netflix Movies And TV Shows - EDA

In [2]:
# Importing all the necessary libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns
import datetime as dt

Loading the dataset using pandas

In [4]:
df = pd.read_csv('netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [5]:
# Dataset shape
df.shape

(8807, 12)

In [6]:
# Data types & non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [7]:
# Summary stats
df.describe()

,release_year
count,8807.000000
mean,2014.180198
std,8.819312
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [8]:
# Checking for duplicates in the entire dataset
print(df.duplicated().sum())
# checking for duplicates in unique id
df['show_id'].duplicated().sum()

0


0

There are no duplicates in the dataset.

In [10]:
# Checking for missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values

director      2634
cast           825
country        831
date_added      10
rating           4
duration         3
dtype: int64

In [11]:
# Calculate percentage of missing data
missing_percentage = (missing_values / len(df)) * 100
missing_percentage.round(2)

director      29.91
cast           9.37
country        9.44
date_added     0.11
rating         0.05
duration       0.03
dtype: float64

### Handling Missing Values

The missing values were found in the following variables:

- **director**: This variable is considered not essential for the analysis as it does not provide significant information.
- **cast**: Due to the large variety of values in this variable, it offers limited actionable insights and will not be included in the analysis.
- **country**: This is an important variable, so we will explore strategies to address missing values, possibly by imputing with the mode.
- **date_added**: Since we need to parse this column, we will convert this to datetime, and leave missing values as NaT.
- **rating**: The missing cases are minimal but this is an important column so we will fill it with the most common value
- **duration**: Upon further investigation, we found that there is an issue of data misalignment with this and `rating` column.

#### Fix cast and director column

In [14]:
df = df.drop(['director', 'cast'], axis=1)
df.columns

Index(['show_id', 'type', 'title', 'country', 'date_added', 'release_year',
       'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

#### Fix missing country values

We replaced all the Nan values in the country column with United States as Netflix was created in the USA and <br> 
every show is aired on Netflix US. So instead of dropping the whole column we just replaced the values in it in order to save our data.

In [17]:
# Replace missing 'country' values with the mode
country_mode = df['country'].mode()[0]
df['country'] = df['country'].fillna(country_mode)

#### Fix missing duration

In [19]:
# Identify rows where duration is NaN and rating looks like duration
duration_missing = df[df['duration'].isnull()]
misaligned_rows = duration_missing[duration_missing['rating'].str.contains('min', na=False)]
misaligned_rows

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description
5541,s5542,Movie,Louis C.K. 2017,United States,"April 4, 2017",2017,74 min,NaN,Movies,"Louis C.K. muses on religion, eternal love, gi..."
5794,s5795,Movie,Louis C.K.: Hilarious,United States,"September 16, 2016",2010,84 min,NaN,Movies,Emmy-winning comedy writer Louis C.K. brings h...
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,United States,"August 15, 2016",2015,66 min,NaN,Movies,The comic puts his trademark hilarious/thought...


In [20]:
# Fix misaligned rows: move rating → duration, set rating to NaN
df.loc[misaligned_rows.index, 'duration'] = df.loc[misaligned_rows.index, 'rating']
df.loc[misaligned_rows.index, 'rating'] = np.nan

In [21]:
# Recheck if any missing durations remain
df['duration'].isnull().sum(), df['rating'].isnull().sum()

(0, 7)

#### Fix missing rating

As we now finally know that the actual missing values are 7 in `rating` column, we can either drop them or replace them. <br>
We have TV-MA which is the most common rating and hence we can replace all these nan values with TV-MA.

In [24]:
df['rating'].value_counts()

rating
TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
Name: count, dtype: int64

In [25]:
df['rating'] = df['rating'].replace(np.nan, 'TV-MA')

In [26]:
df['rating'].isnull().sum()

0

#### Fix date_added column

As I need to parse the `date_added` column in the format "dd-mm-yyyy", the best option would be leave the nulls as NaT for now. <br>
But before parsing, let's see if there are other inconsistencies in the column.

In [29]:
# Find rows with leading/trailing whitespace
df[df['date_added'].str.contains(r'^\s+|\s+$', na=False)]

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description
6079,s6080,TV Show,Abnormal Summit,South Korea,"August 4, 2017",2017,TV-PG,2 Seasons,"International TV Shows, Korean TV Shows, Stand...","Led by a trio of Korean celebs, a multinationa..."
6177,s6178,TV Show,忍者ハットリくん,Japan,"December 23, 2018",2012,TV-Y7,2 Seasons,"Anime Series, Kids' TV","Hailing from the mountains of Iga, Kanzo Hatto..."
6213,s6214,TV Show,Bad Education,United Kingdom,"December 15, 2018",2014,TV-MA,3 Seasons,"British TV Shows, TV Comedies","A history teacher at the posh Abbey Grove, Alf..."
6279,s6280,TV Show,Being Mary Jane: The Series,United States,"July 1, 2017",2016,TV-14,4 Seasons,"Romantic TV Shows, TV Dramas",Ambitious single TV journalist Mary Jane attem...
6304,s6305,TV Show,"Big Dreams, Small Spaces",United Kingdom,"July 26, 2019",2017,TV-G,3 Seasons,"British TV Shows, International TV Shows, Real...",Writer and presenter Monty Don helps England's...
...,...,...,...,...,...,...,...,...,...,...
8539,s8540,TV Show,The Tudors,"Ireland, Canada, United States, United Kingdom","January 8, 2018",2010,TV-MA,4 Seasons,TV Dramas,All the splendor and scandal of England's 16th...
8557,s8558,TV Show,The West Wing,United States,"December 25, 2015",2005,TV-14,7 Seasons,TV Dramas,This powerful political epic chronicles the tr...
8684,s8685,TV Show,Vroomiz,South Korea,"August 1, 2017",2016,TV-Y,3 Seasons,"Kids' TV, Korean TV Shows","For these half-car, half-animal friends, each ..."
8712,s8713,TV Show,Weird Wonders of the World,United Kingdom,"March 31, 2017",2016,TV-PG,2 Seasons,"British TV Shows, Docuseries, Science & Nature TV",From animal oddities and bizarre science to me...


In [30]:
# Clean the column by removing the whitespaces
df['date_added'] = df['date_added'].str.strip()

Now that we removed the whitespaces from the values, we can parse the dates

In [32]:
# Convert to datetime (this will also convert invalid or missing to NaT)
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

In [33]:
# Format the dates to 'dd-mm-yyyy' and replacing NaT values with 'Unavailable'
df['date_added'] = df['date_added'].dt.strftime('%d-%m-%Y')
df['date_added'] = df['date_added'].fillna('Unavailable')

In [34]:
# Check how many values are still missing
print("Missing 'date_added' after parsing:", df['date_added'].isnull().sum())

Missing 'date_added' after parsing: 0


#### Now our dataset is ready for the further analysis and visualization